In [12]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'


import sys
sys.path.append('/Users/wooseongkyun/코드_아카이브/프로젝트2_머신러닝과EDA/binomial_classification/preprocessing_data.py')

In [13]:
#기후 데이터를 가져오기
df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')
df.drop(['강수확률(%)','발표시각','신뢰도'],axis=1,inplace=True)

/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_97355/1886206877.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')


In [14]:
AM_PM_lst=[]
day_lst=[]

date_lst=df['예보시각']
for date in date_lst:
    splited_date= date.split()
    day_lst.append(splited_date[0])
    AM_PM_lst.append(splited_date[-1])

df['예보일']=day_lst
df['오전_오후']=AM_PM_lst

#오전오후를 0과 1값으로 표현하기
is_PM =df['오전_오후']=='오후'
df['오전0_오후1']=is_PM
df['오전0_오후1']= df['오전0_오후1'].astype(int)

#중복된 데이터정보를 나타내는 열들 삭제하기
#계산을 간략화하기 위해 지역에 대한 가중치를 모두 같게함. 그로써 지역에 대한 컬럼을 삭제
df.drop(['예보시각','오전_오후','지역','오전0_오후1'],axis=1,inplace=True)
df.sort_values(by='예보일',ascending=True)

,예보,예보일
0,맑음,2017-01-04
52,맑음,2017-01-04
53,맑음,2017-01-04
65,맑음,2017-01-04
66,맑음,2017-01-04
...,...,...
474642,맑음,2022-01-10
474720,구름많음,2022-01-10
474707,구름많음,2022-01-10
474746,맑음,2022-01-10


In [15]:
#기상 관측 분류를 원-핫 인코딩하기
features=['예보']
encoded_df= pd.get_dummies(data=df,columns=features,prefix=None)
encoded_df


,예보일,예보_구름많고 눈,예보_구름많고 눈/비,예보_구름많고 비,예보_구름많고 비/눈,예보_구름많고 소나기,예보_구름많음,예보_구름조금,예보_맑음,예보_흐리고 눈,예보_흐리고 눈/비,예보_흐리고 비,예보_흐리고 비/눈,예보_흐리고 소나기,예보_흐림
0,2017-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2017-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2017-01-05,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2017-01-05,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2017-01-06,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474755,2022-01-07,0,0,0,0,0,0,0,1,0,0,0,0,0,0
474756,2022-01-07,0,0,0,0,0,0,0,1,0,0,0,0,0,0
474757,2022-01-08,0,0,0,0,0,0,0,1,0,0,0,0,0,0
474758,2022-01-09,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [16]:
#각 지역별 관측된 기상현상값들을 동일한 가중치(1)로 두고 더하여 전국 날씨값을 계산하기
evaluated_df= encoded_df.groupby(by='예보일').sum()
evaluated_df['맑음']=  evaluated_df['예보_맑음']+evaluated_df['예보_구름조금']+evaluated_df['예보_구름많음']
evaluated_df['비']=evaluated_df['예보_구름많고 비']+evaluated_df['예보_구름많고 소나기']+\
                  evaluated_df['예보_흐리고 비']+evaluated_df['예보_흐리고 소나기']
evaluated_df['흐림']=evaluated_df['예보_흐림']
evaluated_df['눈']=evaluated_df['예보_구름많고 눈']+evaluated_df['예보_구름많고 눈/비']+\
                  evaluated_df['예보_구름많고 비/눈']+evaluated_df['예보_흐리고 눈']+\
                  evaluated_df['예보_흐리고 눈/비']+evaluated_df['예보_흐리고 비/눈']


evaluated_df= evaluated_df[['맑음','비','흐림','눈']]
evaluated_df

,맑음,비,흐림,눈
예보일,,,,
2017-01-04,40.0,0.0,0.0,0.0
2017-01-05,71.0,4.0,0.0,5.0
2017-01-06,120.0,0.0,0.0,0.0
2017-01-07,143.0,17.0,0.0,0.0
2017-01-08,148.0,45.0,0.0,7.0
...,...,...,...,...
2022-01-06,127.0,2.0,11.0,0.0
2022-01-07,99.0,0.0,1.0,0.0
2022-01-08,60.0,0.0,0.0,0.0


In [17]:
weather_df=evaluated_df.idxmax(axis=1)
weather_lst = list(weather_df.values)

In [18]:
weather_df=pd.DataFrame(weather_df)
weather_df['기상']=weather_lst
weather_df.drop(0,axis=1,inplace=True)

In [19]:
year_lst=[]
date_lst=list(weather_df.index)
for date in date_lst:
    splited_date= str(date)[0:4]
    year_lst.append(splited_date)

year_lst
weather_df['년도']= year_lst

weather_category=weather_df.groupby('년도').value_counts()
weather_category.loc['2020']
weather_category

년도    기상
2017  맑음    344
      비      18
2018  맑음    358
      비       7
2019  맑음    350
      비      14
      눈       1
2020  맑음    278
      비      61
      흐림     26
      눈       1
2021  맑음    298
      비      40
      흐림     27
2022  맑음     10
dtype: int64

In [20]:
weather_category.to_csv("기상청_전처리된데이터.csv", mode='w')

In [21]:
list(weather_df.index)

['2017-01-04',
 '2017-01-05',
 '2017-01-06',
 '2017-01-07',
 '2017-01-08',
 '2017-01-09',
 '2017-01-10',
 '2017-01-11',
 '2017-01-12',
 '2017-01-13',
 '2017-01-14',
 '2017-01-15',
 '2017-01-16',
 '2017-01-17',
 '2017-01-18',
 '2017-01-19',
 '2017-01-20',
 '2017-01-21',
 '2017-01-22',
 '2017-01-23',
 '2017-01-24',
 '2017-01-25',
 '2017-01-26',
 '2017-01-27',
 '2017-01-28',
 '2017-01-29',
 '2017-01-30',
 '2017-01-31',
 '2017-02-01',
 '2017-02-02',
 '2017-02-03',
 '2017-02-04',
 '2017-02-05',
 '2017-02-06',
 '2017-02-07',
 '2017-02-08',
 '2017-02-09',
 '2017-02-10',
 '2017-02-11',
 '2017-02-12',
 '2017-02-13',
 '2017-02-14',
 '2017-02-15',
 '2017-02-16',
 '2017-02-17',
 '2017-02-18',
 '2017-02-19',
 '2017-02-20',
 '2017-02-21',
 '2017-02-22',
 '2017-02-23',
 '2017-02-24',
 '2017-02-25',
 '2017-02-26',
 '2017-02-27',
 '2017-02-28',
 '2017-03-01',
 '2017-03-02',
 '2017-03-03',
 '2017-03-04',
 '2017-03-05',
 '2017-03-06',
 '2017-03-07',
 '2017-03-08',
 '2017-03-09',
 '2017-03-10',
 '2017-03-